# Evaluating Taekwondo Moves
System to recognize and evaluate video of performed taekwondo moves

##Setup

In [ ]:
import sys
import os
from os.path import exists, join, basename, splitext
import time
import math
import pickle
import numpy as np
import cv2
import json
import random
from google.colab.patches import cv2_imshow
from datetime import datetime

In [ ]:
!pip install -U scikit-learn

     |████████████████████████████████| 22.3MB 5.8MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [ ]:
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

In [ ]:
def zip_folder(zip_folder_name, direc):
  !zip -q -r "$zip_folder_name" "$direc"

In [ ]:
def unzip_folder(zip_folder_name, direc='/content'):
  !unzip "$zip_folder_name" -d "$direc"
  # !unzip -q -l "$zip_folder_name"

In [ ]:
def remove_folder(folder_name):
  !rm -r "$folder_name"

In [ ]:
def get_t():
  return f'{datetime.now()}'.replace(' ', '_').replace(':', '-')

In [ ]:
lu = 'drive/MyDrive/Senior_Research'

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#abbreviations to make folders 
abbreviations = {
    "High block": "hb",
    "Middle block (in-to-out)": "mbio",
    "Middle block (out-to-in)": "mboi",
    "Knife hand block": "khb",
    "Low block": "lb",
    "Punch (middle-level)": "p",
    "High punch (face-level)": "hp",
    "Front kick": "fk",
    "Round kick": "rk",
    "Side kick": "sk"
}

## Skeleton Representation

Get Existing Custom Datasets as of February 23, 2021

In [ ]:
# unzip_folder(f'{lu}/VideoPose3D-2021-02-23.zip')

###Create Custom Datasets

In [ ]:
# git_repo_url = 'https://github.com/facebookresearch/VideoPose3D.git'
# project_name = splitext(basename(git_repo_url))[0]
# if not exists(project_name):
#   # clone and install dependencies
#    !git clone -q --depth 1 $git_repo_url

In [ ]:
!python3 -m pip install --upgrade pip

     |████████████████████████████████| 1.6MB 15.1MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1


In [ ]:
# install dependencies: 
!pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
distutils: /usr/local/include/python3.7/UNKNOWN
sysconfig: /usr/include/python3.7m
distutils: /usr/local/bin
sysconfig: /usr/bin
distutils: /usr/local
sysconfig: /usr
user = False
home = None
root = None
prefix = None
     |████████████████████████████████| 274 kB 20.5 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44074 sha256=a1c36baac9c23b57023177afdea4e9ad2b2d45b4ba02e60e8c3b99c3004517ab
  Stored in directory: /root/.cache/pip/wheels/77/f5/10/d00a2bd30928b972790053b5de0c703ca87324f3fead0f2fd9
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  distutils: /usr/local/lib/python3.7/dist-packages
  sysconfig: /usr/lib/python3.7/site-packag

In [ ]:
# install detectron2: (Colab has CUDA 10.1 + torch 1.8)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
import torch
assert torch.__version__.startswith("1.8")   # need to manually install torch 1.8 if Colab changes its default version
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
# exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
distutils: /usr/local/include/python3.7/UNKNOWN
sysconfig: /usr/include/python3.7m
distutils: /usr/local/bin
sysconfig: /usr/bin
distutils: /usr/local
sysconfig: /usr
user = False
home = None
root = None
prefix = None
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
     |████████████████████████████████| 6.2 MB 568 kB/s 
     |████████████████████████████████| 47 kB 4.4 MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.3.post20210317-py3-none-any.whl size=58542 sha256=89a55c3fe40d9869b7330dd0af6bfe13ffca92cb77c1fb469963ef25bc1a9c21
  Stored in directory: /root/.cache/pip/wheels/a6/02/09/10e3a0150eb92e5ecbee3677a813bffc32a8ec6f876bfe4adf
Successfully built fvcore
  distutils: /usr/local/lib/python3.7/dist-packages
  sysconfig: /usr/lib/python3.7/site-pac

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

*Make sure to upload the pretrained model to the checkpoint folder if you took VideoPose3D straight from GitHub

In [ ]:
def create_train_custom_dataset(videos, side, move, slow): #will create custom dataset with "$prefix"_dataset naming convention
  prefix = download_all_videos(videos, side, move, slow)
  if prefix != None:
    # create an output folder for the 2D estimates
    twod_outputs_folder = f'{prefix}_2d_outputs'
    if not os.path.exists(twod_outputs_folder):
      os.makedirs(twod_outputs_folder)
    !cd VideoPose3D/inference && python3 infer_video_d2.py --cfg COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x.yaml --output-dir ../../"$twod_outputs_folder"  --image-ext mp4 ../../"$prefix"/
    !cd VideoPose3D/data/ && python3 prepare_data_2d_custom.py -i ../../"$twod_outputs_folder" -o "$prefix"_dataset
    return prefix, f'{prefix}_dataset'
  return None, None

In [ ]:
def create_train_custom_dataset_for_given_moves(moves):
  # pdts = {}
  for m in moves:
    prefix, dts = create_train_custom_dataset(videos, 'Left', m, slow=False)
    # pdts[prefix] = dts
    prefix, dts = create_train_custom_dataset(videos, 'Right', m, slow=False)
    # pdts[prefix] = dts
  folder_name = f"{lu}/VideoPose3D-{get_t()}.zip"
  zip_folder(folder_name, "VideoPose3D/")
  # pickle.dump(pdts, open( "pdts.p", "wb" ) 

In [ ]:
def create_custom_dataset(leadup, ip_folder, slow):
  p = os.path.join(leadup, ip_folder)
  if slow == True:
    oup_folder = f'{ip_folder}_slow'
    new_p = os.path.join(leadup, oup_folder)
    if not os.path.exists(new_p):
      os.makedirs(new_p)

    for f in os.listdir(p):
      # print(f)
      vidfn, ext = os.path.splitext(f)
      slow_down_video(os.path.join(p, f), os.path.join(new_p, f), speed=2.0)
  else:
    new_p = p
    oup_folder = ip_folder

  twod_outputs_folder = f'{oup_folder}_2d_outputs'
  if not os.path.exists(twod_outputs_folder):
    os.makedirs(twod_outputs_folder)
  !cd VideoPose3D/inference && python3 infer_video_d2.py --cfg COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x.yaml --output-dir ../../"$twod_outputs_folder"  --image-ext mp4 ../../"$new_p"/
  !cd VideoPose3D/data/ && python3 prepare_data_2d_custom.py -i ../../"$twod_outputs_folder" -o "$oup_folder"_dataset
  return oup_folder, f'{oup_folder}_dataset'

## Get Data

In [ ]:
import pandas as pd
import io

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from googleapiclient.http import MediaIoBaseDownload

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

In [ ]:
df_train = pd.read_excel(f'{lu}/Evaluating_TKD_Video_Submission_Responses_v2.xlsx', index_col=None, na_values=['NA'], usecols=['Timestamp', 'Athlete Name', 'Level Ability', 'Move', 'Side', 'Iteration #', "View", "Upload video here"])
df_train.head()

,Timestamp,Athlete Name,Level Ability,Move,Side,Iteration #,View,Upload video here
0,2020-11-16 21:39:06.037,H.S.,Advanced,High block,Left,1,Diagonal,https://drive.google.com/open?id=1xaXnbMND1HxL...
1,2020-11-16 21:39:46.995,H.S.,Advanced,High block,Left,2,Diagonal,https://drive.google.com/open?id=1dzAncu0Zt2Ds...
2,2020-11-16 21:40:24.293,H.S.,Advanced,High block,Left,3,Diagonal,https://drive.google.com/open?id=1E2StfiV7ENtW...
3,2020-11-16 21:41:52.790,H.S.,Advanced,High block,Left,4,Diagonal,https://drive.google.com/open?id=1-yo-5SzX2jK4...
4,2020-11-16 21:42:50.091,H.S.,Advanced,High block,Left,5,Diagonal,https://drive.google.com/open?id=1gl6WkI0Q2pD_...


In [ ]:
df_full = pd.read_excel(f'{lu}/Evaluating_TKD_Video_Submission_Responses_v4.xlsx', index_col=None, na_values=['NA'], usecols=['Timestamp', 'Athlete Name', 'Level Ability', 'Move', 'Side', 'Iteration #', "View", "Upload video here", "Ideal"])
df_full.head()

,Timestamp,Athlete Name,Level Ability,Move,Side,Iteration #,View,Upload video here,Ideal
0,2020-11-16 21:39:06.037,H.S.,Advanced,High block,Left,1,Diagonal,https://drive.google.com/open?id=1xaXnbMND1HxL...,NaN
1,2020-11-16 21:39:46.995,H.S.,Advanced,High block,Left,2,Diagonal,https://drive.google.com/open?id=1dzAncu0Zt2Ds...,NaN
2,2020-11-16 21:40:24.293,H.S.,Advanced,High block,Left,3,Diagonal,https://drive.google.com/open?id=1E2StfiV7ENtW...,NaN
3,2020-11-16 21:41:52.790,H.S.,Advanced,High block,Left,4,Diagonal,https://drive.google.com/open?id=1-yo-5SzX2jK4...,NaN
4,2020-11-16 21:42:50.091,H.S.,Advanced,High block,Left,5,Diagonal,https://drive.google.com/open?id=1gl6WkI0Q2pD_...,NaN


In [ ]:
df_test = pd.read_excel(f'{lu}/Evaluating_TKD_Video_Submission_Responses_v4_Test.xlsx', index_col=None, na_values=['NA'], usecols=['Timestamp', 'Athlete Name', 'Level Ability', 'Move', 'Side', 'Iteration #', "View", "Upload video here"])
df_test.head()

,Timestamp,Athlete Name,Level Ability,Move,Side,Iteration #,View,Upload video here
0,2021-03-16 15:28:24.060,S.M.,Advanced,High block,Left,0,Front,https://drive.google.com/open?id=1zSjq2YdrSssA...
1,2021-03-16 15:29:04.184,S.M.,Advanced,High block,Left,1,Front,https://drive.google.com/open?id=1h4_jIdjN2AUB...
2,2021-03-16 15:29:46.705,S.M.,Advanced,High block,Left,2,Front,https://drive.google.com/open?id=1GXwsd4FWk4wc...
3,2021-03-16 15:30:29.401,S.M.,Advanced,High block,Left,3,Front,https://drive.google.com/open?id=12dRD-neYFUZP...
4,2021-03-16 15:31:04.491,S.M.,Advanced,High block,Left,4,Front,https://drive.google.com/open?id=13_Aln1IFeXnM...


In [ ]:
def get_file_ids(df):
  videos = {'Front': {'Left': {}, 'Right': {}}, 'Diagonal': {'Left': {}, 'Right': {}}} #view --> side --> moves --> ids

  for index, row in df.iterrows():
    view = videos[row['View']]
    if row['Move'] in view[row['Side']]:
        side = view[row['Side']]
        side[row['Move']].add(row['Upload video here'][row['Upload video here'].find('id') + 3:])
    else:
        side = view[row['Side']]
        side[row['Move']] = set()
        url = row['Upload video here']
        # print(url)
        i = url.find('id') + 3
        # print(i)
        side[row['Move']].add(url[i:])
  return videos

In [ ]:
def get_ideal_file_ids(df):
  move2ideal = {}

  for index, row in df.iterrows():
    ideal = row['Ideal']
    if ideal == "Yes":
      prefix = f"{abbreviations[row['Move']]}_{row['Side'][0].lower()}"
      move2ideal[prefix] = row['Upload video here'][row['Upload video here'].find('id') + 3:]
  return move2ideal

In [ ]:
def print_videos(videos):
  for view in videos:
    print(f'{view}')
    for side in videos[view]:
      print(f'\t{side}')
      moves = videos[view][side]
      for m in moves:
        print(f'\t\t{m} ({len(moves[m])}): {moves[m]}')

In [ ]:
train_videos = get_file_ids(df_train)
print_videos(train_videos)

Front
	Left
		High block (25): {'1Z-epYLjpKyeroDRKdEixoyiN_Ewse4Bw', '1dA5kLSxNcw2i8_ZzeJU1xrRmjZLvLGc7', '1J0ODCUmJ7iwArCeCbEdqDQNNuM-WMXd2', '11OasZyIWhI1kex2dzuEBW92mzlQfX3rB', '13TzOteRlxG3IGkBICiTfrrazdpCBaKQ_', '17hGfxOEs2QAziEqVJLjGbfLN_bVIVrby', '1AWRdXabW6Rq1wtgnWtEbfPikGQqYxzjF', '1xK-4-pgFffXzqqhUSPnhvuFWeiyy5iwk', '19mMLbFSblNv0iZwU52w9jrP5p4SUPWuq', '1ZDfq3xs5gW93CFZmPojTi6xOpM5r9fZ3', '1GRQqaYTZL7S5nKUomTr13zkCmkGT7UP5', '1amIGHLogn5OZt7_PTsnGt6ekl2xkGs4p', '1SYjMssnHbkgXnNoiE-bheZXdL101br6d', '1We_6QsEUHLs3tGiKoINPTk2Dkh2FrkMo', '1BA5NgouC4PETKRN1j-TiiM5xE3NZWsoe', '1mmP7dOcK4iFHaahKuaJidh7PbzSL0iKb', '1bJxIOdkkvfQJR0HYmLKNLY12Xro0o-Kt', '1SlJZ7xlqKHdMNqF85FBNvmz1p7ZfgFqV', '17F_I-7WU3vz_sWMhSRbhRmcYe4wxtMYq', '1jCYV_acB7DTPivznBTwkUBPCxop0D_ea', '1JUuGU9eAn_MlzNEGmYn8JH8HNT03HGtK', '1haZRJ82yi9MDZH3WjU70ctkKqMJOQmAs', '1vVuEHWRInr4IA0Gy0HBsEfgSqkaZC0GX', '1cW7MGyw43JC4pbxc3bCJz1H0Qd-N47gJ', '1KGwcSyTUzYm-vLS-IANhi-5d57FyePKt'}
		Middle block (in-to-out) (25): {'1mLTQin7

In [ ]:
test_videos = get_file_ids(df_test)
print_videos(test_videos)

Front
	Left
		High block (6): {'1GXwsd4FWk4wciNwkQC47UE9alnj5sqvU', '13_Aln1IFeXnMTQjLt1-ynHcpmERyFmYf', '12dRD-neYFUZPbnw_wrS_8HOAM89v0-x4', '1zSjq2YdrSssAZiPtBCp9MS-aXuhP0-8J', '1h4_jIdjN2AUBOSdvbJ5_1fI6mGuc9Cml', '1U8JFG8O3NZG0-DAjzxOihvFxuxL1Xo-8'}
		Middle block (in-to-out) (5): {'1xkAhj14URQdtqx73zCBwrTZ8gmPB0IvX', '1uHBalHc8OaSn-r8emsrNBcUCYHUdbrGm', '1gsbq4KFJNLR-SbwwWGihPDyakzwEqoMi', '1u2wDINjE5ziTOutGhCHXxo8_29w9OoN2', '1fa1iaFnmislHXKtoPX0QWTN7FD-ErPNg'}
		Middle block (out-to-in) (6): {'1IpyBKYpFspYRefXgzyHJ6C6IWez12UOY', '1ae19Fzi_c8N1cxPtJSoVC44V04MrKsmV', '1PNJhTnVIG9HfXGywFg86w3MJ7auF480E', '1LtoMuGyb_18CG07jV82_pxv-Kc6HY0UY', '1pGis5yZrO9pzp8vPX3CrVhCJgfNEUHcJ', '1aVO-LoGJT-ZD3yzBVNhGwgKNyPwd0G10'}
		Knife hand block (6): {'1EDOTLHj32G2BCQGXOFHydrjU1dqs1Uzi', '1AoK_rpN7yQse9oKmOIG_mAvGUXCK3V05', '1Y8qxMXQjdVmMJKsljtxsrYsL6hjQfKxe', '1YqT68bQbpU2qbDz1NkY5mELTXb4JZ9Ps', '1zSrTk834ITUMdhRvs5uEjN5eBWIykE_X', '1Aq3FD8ZG-9IhsArCWTHJm4vv91XJ-Cdf'}
		Low block (5): {'1qtFwBVv

In [ ]:
def slow_down_video(input_name, output_name, speed=3.0):
  !ffmpeg -i "$input_name" -filter:v "setpts=$speed*PTS" "$output_name"

In [ ]:
def naming_convention(c, prefix, view, count):
  if c == 'regular':
    filename = prefix + f'{view[0].lower()}_{count}.mp4'
  else:
    filename = prefix + '_ideal.mp4'
  return filename

In [ ]:
def download_video_helper(file_id, dir, prefix, view, count, convention):
  request = drive_service.files().get_media(fileId=file_id)
  fh = io.BytesIO()
  downloader = MediaIoBaseDownload(fh, request)
  done = False
  while done is False:
      status, done = downloader.next_chunk()
      # print("Download %d%%." % int(status.progress() * 100))
  fh.seek(0)

  filename = naming_convention(convention, prefix, view, count)

  with open(os.path.join(dir, filename), 'wb') as f:
    f.write(fh.read())
    f.close()

In [ ]:
def download_all_videos_helper(videos, dir, prefix, view, side, move):
  count = 0
  for file_id in videos[view][side][move]:
    # print(file_id)
    download_video_helper(file_id, dir, prefix, view, count, 'regular')
    count += 1
  print(f'Downloaded all {view}-view videos of {side}-side {move} moves')    

In [ ]:
def download_all_video_views(dir, prefix, videos, side, move):
  if move in videos['Front'][side]:
    download_all_videos_helper(videos, dir, prefix, 'Front', side, move)
  if move in videos['Diagonal'][side]:
    download_all_videos_helper(videos, dir, prefix, 'Diagonal', side, move)

In [ ]:
def download_all_videos(leadup, videos, side, move, slow, dts_id=None):  
  if slow:
    if dts_id == None:
      dir = os.path.join(leadup, f'{abbreviations[move]}_{side[0].lower()}_init_')
      print(dir)
      if not os.path.exists(dir): #make the initial videos folder
        os.makedirs(dir)
  else:
    if dts_id == None:
      dir = os.path.join(leadup, f'{abbreviations[move]}_{side[0].lower()}')
    else:
      dir = os.path.join(leadup, f'{abbreviations[move]}_{side[0].lower()}_{dts_id}')
  
  if dts_id == None:
    fdir = os.path.join(leadup, f'{abbreviations[move]}_{side[0].lower()}')
  else:
    fdir = os.path.join(leadup, f'{abbreviations[move]}_{side[0].lower()}_{dts_id}')
  
  if not os.path.exists(fdir):
    os.makedirs(fdir) #make the final videos folder
    prefix = f'{abbreviations[move]}_{side[0].lower()}'
    print(prefix)
    
    download_all_video_views(dir, prefix, videos, side, move)

    if slow:
      for f in os.listdir(dir):
        vidfn, ext = os.path.splitext(f)
        slow_down_video(os.path.join(dir,f), os.path.join(fdir, f'{vidfn}.mp4'))

    return prefix
  else:
    return None

## Pose Classification

In [ ]:
# moves = ["High block", "Middle block (in-to-out)", "Middle block (out-to-in)", "Knife hand block", "Low block", "Punch (middle-level)", "High punch (face-level)", "Front kick", "Round kick", "Side kick"]
# moves = ["High block", "Middle block (in-to-out)", "Middle block (out-to-in)", "Knife hand block", "Low block", "Punch (middle-level)", "High punch (face-level)"]
# moves = ["Front kick", "Round kick", "Side kick"]

In [ ]:
def reconstruct_video_helper(path, threed_outputs_folder, custom_dataset, video_fn):
  p = os.path.join(path, video_fn)
  print(p)
  # print(f'cd VideoPose3D/ && python run.py -d custom -k {custom_dataset} -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject {video_fn}.mp4 --viz-action custom --viz-camera 0 --viz-video ../{p}.mp4 --viz-output ../{threed_outputs_folder}/{video_fn}_si.mp4 --viz-export ../{threed_outputs_folder}/{video_fn}_si --viz-size 6')
  !cd VideoPose3D/ && python run.py -d custom -k "$custom_dataset" -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject "$video_fn".mp4 --viz-action custom --viz-camera 0 --viz-video ../"$p".mp4 --viz-output ../"$threed_outputs_folder"/"$video_fn"_si.mp4 --viz-export ../"$threed_outputs_folder"/"$video_fn"_si --viz-size 6

In [ ]:
def reconstruct_videos(leadup, prefix, dts, folder=None):
  # path = f'../{prefix}/'
  # print(f'Dataset name: {dts}')
  if folder is None:
    p = os.path.join(leadup, prefix)
    threed_outputs_folder = os.path.join(leadup, f'{prefix}_3d_outputs')
  else:
    p = os.path.join(leadup, folder)
    threed_outputs_folder = os.path.join(leadup, f'{folder}_3d_outputs')

  if not os.path.exists(threed_outputs_folder):
    os.makedirs(threed_outputs_folder)

  for f in os.listdir(p):
    print(f)
    vidfn, ext = os.path.splitext(f)
    reconstruct_video_helper(p, threed_outputs_folder, dts, vidfn)
  return threed_outputs_folder

###Generate Vocabulary Poses with K-means clustering

In [ ]:
hand_techniques = ["High block", "Middle block (in-to-out)", "Middle block (out-to-in)", "Knife hand block", "Low block", "Punch (middle-level)", "High punch (face-level)"]
kicking_techniques = ["Front kick", "Round kick", "Side kick"]

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
#to calculate angles between joints
#note that math returns angles in radians

h36m_num_joints = 17
h36m_joints = {
    0: [7,8,9],
    1: [14,8,11],
    2: [8,14,15],
    3: [14,15,16],
    4: [8,11,12],
    5: [11,12,13],
    6: [7,1,2],
    7: [1,2,3],
    8: [7,4,5],
    9: [4,5,6],
    10: [16,7,13],
    11: [3,7,6],
    12: [8,7,16],
    13: [8,7,13]
}

#note that the returned angle will be in radians
#where coordinates is a numpy array
def calc_angle(angle_jgroup, coordinates):
    angles = np.array([])
    for i in angle_jgroup:
        s,c,e = angle_jgroup[i]
        s_hat = coordinates[s] - coordinates[c]
        c_hat = coordinates[c] - coordinates[c]
        e_hat = coordinates[e] - coordinates[c]

        dot_product = s_hat@e_hat
        alpha = math.acos((dot_product)/(np.linalg.norm(s_hat)*np.linalg.norm(e_hat)))

        angles = np.append(angles, alpha)
    return angles

#given filename with frame.json, gets angles
def get_frame_angles(frame):
    ang = calc_angle(h36m_joints, frame)
    return ang

#rerturns a series of 14d arrays (angles in radians)
def get_angles(data):
    ang = np.empty((len(data),len(h36m_joints)))
    for i, d in enumerate(data):
        f_ang = get_frame_angles(d)
        ang[i] = f_ang
    return ang

In [ ]:
#input - a bunch of NumPy archives of a given move
def get_cluster_centroids(k, ips):
  X = get_angles(np.load(ips[0]))
  i = 1
  while i < len(ips):
      data = get_angles(np.load(ips[i]))
      X = np.concatenate((X, data))
      i+=1
  # print(X.shape)
  kmeans = KMeans(n_clusters=k, random_state=0).fit(X)

  # print(kmeans.labels_)
  cc = kmeans.cluster_centers_
  # print(cc)
  return kmeans

In [ ]:
def get_ips(leadup, prefix):
  threed_outputs_folder =  os.path.join(leadup, f'{prefix}_3d_outputs')
  ips = []
  for f in os.listdir(threed_outputs_folder):
      vidfn, ext = os.path.splitext(f)
      # print(ext)
      if ext == '.npy':
        ips.append(os.path.join(threed_outputs_folder, f))
  print(ips)
  return ips

In [ ]:
def get_all_cluster_centroids_helper(leadup, videos, k, side, move, ks, dl=False):
  if dl == True:
    prefix = download_all_videos(leadup, videos, side, move, slow=False)
    dts = f'{prefix}_dataset'
    threed_outputs_folder = reconstruct_videos(leadup, prefix, dts)
  else:
    prefix = f'{abbreviations[move]}_{side[0].lower()}'
  ips = get_ips(leadup, prefix)
  kmeans = get_cluster_centroids(k, ips)
  ks[prefix] = kmeans
  return kmeans

In [ ]:
def get_all_cluster_centroids(leadup, videos, k, moves, ks, ks_path, dl=False):
  for m in moves:
    print(f'{abbreviations[m]}_r')
    get_all_cluster_centroids_helper(leadup, videos, k, "Right", m, ks, dl=dl)
    pickle.dump(ks, open(ks_path, "wb" ))
    print(f'{abbreviations[m]}_l')
    get_all_cluster_centroids_helper(leadup, videos, k, "Left", m, ks, dl=dl)
    pickle.dump(ks, open(ks_path, "wb" ))

In [ ]:
# unzip_folder(os.path.join(lu, 'TrainSet_NPYs&MP4s.zip'))

In [ ]:
# ks = {}
hand_vp = 10
kick_vp = 12
# ks_path = os.path.join(lu, f'ks-h{hand_vp}k{kick_vp}.p')

In [ ]:
# get_all_cluster_centroids('', train_videos, hand_vp, hand_techniques, ks, ks_path)
# get_all_cluster_centroids('', train_videos, kick_vp, kicking_techniques, ks, ks_path)

###KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
def get_data(path_to_ks, suffix, new_identifiers=False):
    ks = pickle.load(open(path_to_ks, 'rb'))

    num_angles = 14
    total_poses = (hand_vp * 2 * len(hand_techniques)) + (kick_vp * 2 * len(kicking_techniques))

    X = np.empty((total_poses, num_angles))
    y = np.array([])

    if new_identifiers == True:
        identifier2move = {}
        move2identifier = {}
        identifiers = [i for i in range(len(ks.values()))]
        random.shuffle(identifiers)
    else:
        identifier2move, move2identifier = pickle.load(open(os.path.join(lu, f'moveids.p'), 'rb'))

    count = 0
    for move in ks:
        if new_identifiers == True:
            moveid = identifiers.pop()
            identifier2move[moveid] = move
            move2identifier[move] = moveid
        else:
            moveid = move2identifier[move]
        vocab_poses = ks[move].cluster_centers_
        for pose in vocab_poses:
            X[count] = pose #where X is the collection of vocab poses that map
            y = np.append(y, [moveid])
            count += 1
    np.savez(os.path.join(lu, suffix), X, y)
    if new_identifiers == True:
        pickle.dump((identifier2move, move2identifier), open(os.path.join(lu, f'moveids.p'), 'wb'))
    return X,y, identifier2move, move2identifier

In [ ]:
def classify_videos(ips, tech, ks_name, new_ids=False):
    path_to_ks = os.path.join(lu, ks_name)
    suffix = path_to_ks[path_to_ks.find('-')+1:path_to_ks.find('.p')]

    if not os.path.exists(os.path.join(lu, f'{suffix}.npz')):
        X, y, identifier2move, move2identifier = get_data(path_to_ks, suffix, new_identifiers=new_ids)
    else:
        npzfile = np.load(os.path.join(lu, f'{suffix}.npz'))
        X,y = npzfile['arr_0'], npzfile['arr_1']
        identifier2move, move2identifier = pickle.load(open(os.path.join(lu, f'moveids.p'), 'rb'))

    # neigh = KNeighborsClassifier(n_neighbors=5)
    neigh = KNeighborsClassifier(n_neighbors=10)
    # neigh = KNeighborsClassifier(n_neighbors=12)
    neigh.fit(X, y)

    final_res = []
    for i, ip in enumerate(ips):
        if tech == None or tech[i] == 'k':
          num_vp = int(suffix[suffix.find('k')+1:])
        elif tech[i] == 'h':
            num_vp = int(suffix[suffix.find('h')+1:suffix.find('k')])

        sample_kmeans = get_cluster_centroids(num_vp, [ip])

        res = []
        for pose in sample_kmeans.cluster_centers_:
            res.append(neigh.predict([pose]))

        tally = {}
        for r in res:
            m = identifier2move[r[0]]
            if m not in tally:
                tally[m] = 1
            else:
                tally[m] += 1

        # for w in sorted(tally, key=tally.get, reverse=True):
        #     print(w, tally[w])
            # final_res.append(tally.values())
        maxvote_move = max(tally, key=tally.get)
        final_res.append((maxvote_move, f'{tally[maxvote_move]}/{sum(tally.values())}'))
    return final_res

In [ ]:
def pose_classification_helper(leadup, m, moves_type, dts_id, side, ks_name, pr, acc_writer, dl=False):
  if dl == True:
    prefix = download_all_videos(leadup, test_videos, side, m, False, dts_id)
    oup_folder, dts = create_custom_dataset(leadup, f'{prefix}_{dts_id}', slow=False)
    reconstruct_videos(leadup, f'{prefix}_{dts_id}', dts)
  else:
    prefix = f'{abbreviations[m]}_{side[0].lower()}'
  
  ips = get_ips(leadup, f'{prefix}_{dts_id}')
  tech = [moves_type]*len(ips)
  
  res = classify_videos(ips, tech, ks_name)
  for ip, fr in zip(ips, res):
    print(f'{os.path.basename(ip)}\t|\t{fr[0]}\t{fr[1]}')
    acc_writer.write(f'{os.path.basename(ip)}\t|\t{fr[0]}\t{fr[1]}\n')
  
  misclass = 0
  for (elem, vote) in res:
    if elem != prefix:
      misclass += 1
    pr.append((elem, prefix))
  acc = 1-(misclass/len(ips))
  print(f'Accuracy: {acc}')
  acc_writer.write(f'Accuracy: {acc}\n')
  return acc

In [ ]:
def pose_classification(leadup, moves, moves_type, dts_id, ks_name, pr, version, dl=False):
  accuracies = []
  acc_writer = open(os.path.join(lu, f"pose_class_res_{ks_name[ks_name.find('-')+1:ks_name.find('.')]}_v{version}.txt"), "a")
  for m in moves:
    acc = pose_classification_helper(leadup, m, moves_type, dts_id, 'Left', ks_name, pr, acc_writer, dl=dl)
    accuracies.append(acc)
    acc = pose_classification_helper(leadup, m, moves_type, dts_id, 'Right', ks_name, pr, acc_writer, dl=dl)
    accuracies.append(acc)
  avg_accuracy = sum(accuracies)/len(accuracies)
  print(f'Average Accuracy: {avg_accuracy}')
  acc_writer.write(f'Average Accuracy: {avg_accuracy}\n\n')
  acc_writer.close()
  return 

In [ ]:
def get_f1(prefix, pr): #searching in pr for this prefix
  tp = 0
  fp = 0
  tn = 0
  fn = 0
  for pred, res in pr:
    if pred == res == prefix:
      tp += 1
    elif pred == prefix and res != prefix:
      fp += 1
    elif pred != prefix and res != prefix:
      tn += 1
    else:
      fn += 1
  precision = tp/(tp + fp)
  recall = tp/(tp + fn)
  f1 = 2 * (precision * recall)/(precision + recall)
  return f1

In [ ]:
# !cd "$lu/TestSet" && zip -q -r "TestSet_NPYs&MP4s.zip" ''

In [ ]:
unzip_folder(os.path.join(lu, 'TestSet_NPYs&MP4s.zip'))

In [ ]:
ks_name = f'ks-h{hand_vp}k{kick_vp}.p'
pr = []
pose_classification('TestSet', hand_techniques, 'h', 'test', ks_name, pr, version=4, dl=False)
pose_classification('TestSet', kicking_techniques, 'k', 'test', ks_name, pr, version=4, dl=False)

In [ ]:
for n in abbreviations:
  ab = abbreviations[n]
  get_f1(ab)

## Temporal Alignment (DTW)

In [ ]:
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw

In [ ]:
def get_ideal_videos(leadup, dts_id='ideal'):
  move2ideal = get_ideal_file_ids(df_full)
  print(move2ideal)
  dir = os.path.join(leadup, dts_id)
  if not os.path.exists(dir): #make the initial videos folder
    os.makedirs(dir)
  for prefix in move2ideal:
    download_video_helper(move2ideal[prefix], dir, prefix, None, None, dts_id)
  print("Downloaded all ideal videos")
  oup_folder, dts = create_custom_dataset(leadup, dts_id, slow=False)
  reconstruct_videos(leadup, None, dts, dts_id)

In [ ]:
#get_ideal_videos()
unzip_folder(os.path.join(lu, 'IdealSet_NPYs&MP4s.zip'))

In [ ]:
def dtw_compare_samples_helper(ips, oup):
  d = []
  y = get_angles(np.load(oup))
  for i in ips:
    X = get_angles(np.load(i))
    distance, path = fastdtw(X, y, dist=euclidean)
    d.append(distance)
  return d

In [ ]:
def dtw_compare_samples(leadup, prefix, ips):
  oup = os.path.join(leadup, f'{prefix}_ideal_si.npy')
  d = dtw_compare_samples(ips, oup)
  print(d)
  return d

In [ ]:
#sample test
# prefix = 'mboi_l'
# ips = get_ips('TestSet', f'{prefix}_test')
# eval_helper('ideal_3d_outputs', prefix, ips)

['TestSet/mboi_l_test_3d_outputs/mboi_lf_3_si.npy', 'TestSet/mboi_l_test_3d_outputs/mboi_lf_2_si.npy', 'TestSet/mboi_l_test_3d_outputs/mboi_lf_5_si.npy', 'TestSet/mboi_l_test_3d_outputs/mboi_lf_0_si.npy', 'TestSet/mboi_l_test_3d_outputs/mboi_ld_3_si.npy', 'TestSet/mboi_l_test_3d_outputs/mboi_ld_1_si.npy', 'TestSet/mboi_l_test_3d_outputs/mboi_lf_4_si.npy', 'TestSet/mboi_l_test_3d_outputs/mboi_ld_0_si.npy', 'TestSet/mboi_l_test_3d_outputs/mboi_lf_1_si.npy', 'TestSet/mboi_l_test_3d_outputs/mboi_ld_2_si.npy', 'TestSet/mboi_l_test_3d_outputs/mboi_ld_4_si.npy']
[118.37981110534696, 121.87493197053644, 120.64686262188513, 125.13725581337525, 114.20706484106677, 117.08114387773459, 120.16952144225588, 113.44553619324816, 127.01499183767692, 110.15551499086516, 113.47674116237499]


## GUI

In [ ]:
!chmod u+x "$lu"/convert2mp4.sh

In [ ]:
def convert2mp4(base):
  !cd "$base" && ./../"$lu"/convert2mp4.sh

In [ ]:
from IPython.display import clear_output
def evaluate_video():
  file = input('Enter the name of your video (ex. i/am/a/boss/test.mp4): ') #assume in directory
  lu_pipe, tail = os.path.split(file)
  vidfn, ext = os.path.splitext(tail)

  if ext != ".mp4":
    convert2mp4(lu_pipe)

  ip_f_path = os.path.join(lu_pipe, vidfn)
  if not os.path.exists(ip_f_path):
    os.makedirs(ip_f_path)
  !mv "$file" "$ip_f_path"
  
  oup_f, dts = create_custom_dataset(lu_pipe, vidfn, slow=False)
  threed_outputs_folder_path = reconstruct_videos(lu_pipe, None, dts, oup_f)
  show_local_mp4_video(f'{threed_outputs_folder_path}/{vidfn}_si.mp4')
  ips = get_ips(lu_pipe, oup_f)
  res = classify_videos(ips, None, ks_name)
  move = res[0][0]
  confidence = res[0][1]
  confidence = int(confidence[:confidence.find('/')])/int(confidence[confidence.find('/')+1:])
  
  clear_output()
  
  print(f"Your video was classified as a {move} with {confidence*100}% confidence")
  d = eval_helper(os.path.join(lu, 'ideal_3d_outputs'), move, ips)
  print(f"Relative to the exemplar, your score is {d}. The closer to 0, the better your execution. Try again to get closer to 0.")

In [ ]:
evaluate_video()